https://www.youtube.com/watch?v=6DGYj1EEWOw&t=240s

https://github.com/rohan-paul/LLM-FineTuning-Large-Language-Models/blob/main/Mistral_FineTuning_with_PEFT_and_QLORA.ipynb

https://github.com/brevdev/notebooks/blob/main/mistral-finetune-own-data.ipynb

I apparently need a gpu for quantization - will return

In [1]:
pip install -q -U peft bitsandbytes accelerate trl datasets


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dataclasses import dataclass, field
from datasets import load_dataset
from typing import Optional
import torch
from peft import LoraConfig
from peft import AutoPeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer

In [4]:
train_dataset = load_dataset("json", data_files="NewsDataTraining.json")
test_dataset = load_dataset("json", data_files="NewsDataTesting.json")

Generating train split: 50 examples [00:00, 4229.41 examples/s]
Generating train split: 10 examples [00:00, 3353.57 examples/s]


In [9]:
base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

config.json: 100%|██████████| 571/571 [00:00<00:00, 319kB/s]


RuntimeError: No GPU found. A GPU is needed for quantization.